In [1]:
import numpy as np 
import pandas as pd 

In [3]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 
    'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 
    'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 
    'sqft_living':float, 'floors':str, 'condition':int, 'lat':float, 'date':str, 
    'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}



In [18]:
sales=pd.read_csv('kc_house_data.csv',dtype=dtype_dict)


In [6]:
def get_numpy_data(data_sframe,features,output):
    # add a constant column to an SFrame 
    data_sframe['constant'] =1 
    features = ['constant'] + features 
    # select the columns of data_SFrame given by the ‘features’ list into the SFrame ‘features_sframe’
    features_sframe =data_sframe[features]
    # this will convert the features_sframe into a numpy matrix with GraphLab Create >= 1.7!!
    features_matrix = features_sframe
    
    # assign the column of data_sframe associated with the target to the variable ‘output_sarray’
    output_sarray=data_sframe[output]
    # this will convert the SArray into a numpy array
    output_array = output_sarray
    return(features_matrix,output_array)


Also, copy and paste the predict_output() function to compute the predictions for an entire matrix of features given the matrix and the weights:

In [7]:
def predict_output(feature_matrix,weights):
    predictions = np.dot(feature_matrix,weights)
    return(predictions)

In [8]:
x=np.array([[  1.00000000e+00,   1.18000000e+03,   3.00000000e+00]])
y=np.array([1., 4., 1.])
pred = predict_output(x,y)
pred

array([ 4724.])

#5. In the house dataset, features vary wildly in their relative magnitude: ‘sqft_living’ is very large overall compared to ‘bedrooms’, for instance. As a result, weight for ‘sqft_living’ would be much smaller than weight for ‘bedrooms’. This is problematic because “small” weights are dropped first as l1_penalty goes up.

To give equal considerations for all features, we need to normalize features as discussed in the lectures: we divide each feature by its 2-norm so that the transformed feature has norm 1.

#6. Write a short function called ‘normalize_features(feature_matrix)’, which normalizes columns of a given feature matrix. The function should return a pair ‘(normalized_features, norms)’, where the second item contains the norms of original features. As discussed in the lectures, we will use these norms to normalize the test data in the same way as we normalized the training data.



In [9]:
X = np.array([[3.,5.,8.],[4.,12.,15.]])
print X

[[  3.   5.   8.]
 [  4.  12.  15.]]


In [10]:
norms = np.linalg.norm(X, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
print norms
# axis =0 means col wise sqrt(3^2 + 4^2), sqrt(5^2 + 12^2 ) ,sqrt(8^2 + 15^2 )
# axis =1 means rows wise sqrt(3^2+5^2+8^2), sqrt(4^2+12^2+15^2)

[  5.  13.  17.]


In [11]:
print X / norms # gives [X[:,0]/norm(X[:,0]), X[:,1]/norm(X[:,1]), X[:,2]/norm(X[:,2])]
# [3/5 ,5/13,8/17 ] so is for 2nd row

[[ 0.6         0.38461538  0.47058824]
 [ 0.8         0.92307692  0.88235294]]


In [144]:
def normalize_features(feature_matrix):
    #print feature_matrix
    norms = np.linalg.norm(feature_matrix, axis=0)
    normalized_features = (feature_matrix / norms )
    return (normalized_features,norms)
    

To test the function, run the following:

In [15]:
feature_matrix=np.array([[3.,6.,9.],[4.,8.,12.]])
norms = np.linalg.norm(feature_matrix,axis=0)
normalized_features = feature_matrix/norms 
print normalized_features
print ''
print norms


[[ 0.6  0.6  0.6]
 [ 0.8  0.8  0.8]]

[  5.  10.  15.]


In [124]:
feature_matrix=np.array([[3.,6.,9.],[4.,8.,12.]])
features,norms = normalize_features(feature_matrix)
print features
# should print
# [[ 0.6  0.6  0.6]
#  [ 0.8  0.8  0.8]]
print norms
type(features)


[[ 0.6  0.6  0.6]
 [ 0.8  0.8  0.8]]
[  5.  10.  15.]


numpy.ndarray

# Implementing Coordinate Descent with normalized features
 
We seek to obtain a sparse set of weights by minimizing the LASSO cost function

SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).
(By convention, we do not include w[0] in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use coordinate descent: at each iteration, we will fix all weights but weight i and find the value of weight i that minimizes the objective. That is, we look for

argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]
where all weights other than w[i] are held to be constant. We will optimize one w[i] at a time, circling through the weights multiple times.

Pick a coordinate i
Compute w[i] that minimizes the cost function SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|)
Repeat Steps 1 and 2 for all coordinates, multiple times
For this notebook, we use cyclical coordinate descent with normalized features, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:

       ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
w[i] =  ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
       
       └ (ro[i] - lambda/2)     if ro[i] > lambda/2
where

ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ].
Note that we do not regularize the weight of the constant feature (intercept) w[0], so, for this weight, the update is simply:

w[0] = ro[i]


# Effect of L1 penalty

#9. Consider a simple model with 2 features: ‘sqft_living’ and ‘bedrooms’. The output is ‘price’.
	•	First, run get_numpy_data() (or equivalent) to obtain a feature matrix with 3 columns (constant column added). Use the entire ‘sales’ dataset for now.
	•	Normalize columns of the feature matrix. Save the norms of original features as ‘norms’.
	•	Set initial weights to [1,4,1].
	•	Make predictions with feature matrix and initial weights.
	•	Compute values of ro[i], where
    
    ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]



In [19]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)

In [21]:
simple_feature_matrix.head(2)

,constant,sqft_living,bedrooms
0,1,1180.0,3.0
1,1,2570.0,3.0


In [22]:
simple_feature_matrix,norms = normalize_features(simple_feature_matrix)

In [23]:
simple_feature_matrix.head(2)

,constant,sqft_living,bedrooms
0,0.006802,0.003530,0.005836
1,0.006802,0.007689,0.005836


In [26]:
#Set initial weights to [1,4,1]. 

weights = np.array([1., 4., 1.])
print weights[1]

# Make predictions with feature matrix and initial weights. 
prediction = predict_output(simple_feature_matrix,weights)
prediction

4.0


array([ 0.02675867,  0.04339256,  0.01990703, ...,  0.02289873,
        0.03178473,  0.02289873])

In [36]:
# Compute values of ro[i], where
# ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]

simple_feature_matrix.iloc[:,0].head(2) # all rows 0th col ,here we are extracting the features which is cols 
simple_feature_matrix.iloc[:,1].head(2) # all rows 1st col 


0    0.003530
1    0.007689
Name: sqft_living, dtype: float64

In [45]:
num_features = simple_feature_matrix.shape[1]
print num_features
r0={}
for i in range(num_features):
    if(i==0):
        r0[i]=weights[0]
    else:
        r0[i]=(simple_feature_matrix.iloc[:,i]*(output - prediction + (weights[i]*simple_feature_matrix.iloc[:,i]))).sum()

3


In [46]:
r0

{0: 1.0, 1: 87939470.82325152, 2: 80966698.66623905}

#10. Quiz Question: Recall that, whenever ro[i] falls between -l1_penalty/2 and l1_penalty/2, the corresponding weight w[i] is sent to zero. Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. What range of values of l1_penalty would not set w[1] zero, but would set w[2] to zero, if we were to take a step in that coordinate?


In [47]:
# w[i]=0 ,if -lambda/2 <= ro[i] <= lambda/2
# so for w[2] =0 , multiply 2 on both sides of above i.e -lambda <= 2*ro[i] <=lambda 
# i.e lambda >= -2*ro[i] and lambda <=2*ro[i] i.e -2*ro[i] <= lambda <= 2*ro[i]
print (-2*r0[2],2*r0[2] )


# What range of values of l1_penalty would not set w[1] zero
'''
      ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)     if ro[i] > lambda/2
'''
# for lambda not to be zero ro[i] < -lambda/2 i.e -2*ro[i] > lambda  and ro[i] > lambda/2 i.e 2*ro[i] > lambda
print (-2*r0[1],2*r0[1])
# i.e -175878941.54598215 > stricty greater than lambda and 175878941.54598215  > lambda 

(-161933397.3324781, 161933397.3324781)
(-175878941.64650303, 175878941.64650303)


What range of values of l1_penalty would set both w[1] and w[2] to zero, if we were to take a step in that coordinate?

In [51]:
print (-2*r0[1],2*r0[1] )
print (-2*r0[2],2*r0[2] )
# for both of them to be zero lambda shd be within the above ranges 
diff =abs((r0[1]*2) - (r0[2]*2))
print diff

(-175878941.64650303, 175878941.64650303)
(-161933397.3324781, 161933397.3324781)
13945544.314


# Single Coordinate Descent Step

#12. Using the formula above, implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.


In [78]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix,weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = sum(feature_matrix.iloc[:,i]*(output - prediction + (weights[i]*feature_matrix.iloc[:,i])))
    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i 
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + l1_penalty/2.
    elif ro_i > l1_penalty/2.:
        new_weight_i =  ro_i - l1_penalty/2.
    else:
        new_weight_i = 0.
    
    return new_weight_i

In [80]:
# should print 0.425558846691
import math
x=np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]])
y=np.array([1., 1.])
z=np.array([1., 4.])
print x ,y , z
print np.dot(x,z)
#print lasso_coordinate_descent_step(1, x, y, z , 0.1)

[[ 0.83205029  0.31622777]
 [ 0.5547002   0.9486833 ]] [ 1.  1.] [ 1.  4.]
[ 2.09696136  4.34943339]


# Cyclical coordinate descent
Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:

As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to step 1.
Return weights

IMPORTANT: when computing a new weight for coordinate i, make sure to incorporate the new weights for coordinates 0, 1, ..., i-1. One good way is to update your weights variable in-place. See following pseudocode for illustration.

for i in range(len(weights)):
    old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
    # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
    #     and old values for weight[i], ..., weight[d-1]
    weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)

    # use old_weights_i to compute change in coordinate
    ...

In [73]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    converged = False
    weights = initial_weights
    while not converged:
        weights_diff=[]
        for i in range(len(weights)):
            old_weights_i = weights[i]
            weights[i] = lasso_coordinate_descent_step(i,feature_matrix,output,weights,l1_penalty)
            change = abs(old_weights_i - weights[i] )
            weights_diff.append(change)
        if(sum(weights_diff) < tolerance):
            converged=True
    return weights

#14. Let us now go back to the simple model with 2 features: ‘sqft_living’ and ‘bedrooms’. 

Using ‘get_numpy_data’ (or equivalent), extract the feature matrix and the output array from from the house dataframe. Then normalize the feature matrix using ‘normalized_features()’ function.
Using the following parameters, learn the weights on the sales dataset.
	
    
    •	Initial weights = all zeros
	•	L1 penalty = 1e7
	•	Tolerance = 1.0


In [56]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0


In [112]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

In [125]:
print simple_feature_matrix.head(2)
type(simple_feature_matrix)


   constant  sqft_living  bedrooms
0         1       1180.0       3.0
1         1       2570.0       3.0


pandas.core.frame.DataFrame

In [61]:
print normalized_simple_feature_matrix.shape
print simple_norms
normalized_simple_feature_matrix.head(2)

(21613, 3)
[  1.47013605e+02   3.34257264e+05   5.14075870e+02]


,constant,sqft_living,bedrooms
0,0.006802,0.003530,0.005836
1,0.006802,0.007689,0.005836


In [81]:
#Then, run your implementation of LASSO coordinate descent:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

In [84]:
weights[0],weights[1],weights[2]

(21624995.713070214, 63157249.262907743, 0.0)

In [75]:
#15. Quiz Question: What is the RSS of the learned model on the normalized dataset?
prediction = predict_output(normalized_simple_feature_matrix, weights)
RSS = ((prediction-output)**2).sum()

print RSS



1.63049246002e+15


In [85]:
#16. Quiz Question: Which features had weight zero at convergence?
zip(normalized_simple_feature_matrix,weights)

[('constant', 21624995.713070214),
 ('sqft_living', 63157249.262907743),
 ('bedrooms', 0.0)]

# Evaluating LASSO fit with more features
#17. Let us split the sales dataset into training and test sets. If you are using GraphLab Create, call ‘random_split’ with .8 ratio and seed=0. Otherwise, please down the corresponding csv files from the downloads section.


In [142]:
train_data = pd.read_csv('kc_house_train_data.csv')
test_data = pd.read_csv('kc_house_test_data.csv')
train_data.floors.dtype

dtype('float64')

In [113]:
train_data.head(2)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,constant
0,7129300520,20141013T000000,221900.0,3.0,1.00,1180.0,5650,1,0,0,...,1180,0,1955,0,98178,47.5112,-122.257,1340.0,5650.0,1
1,6414100192,20141209T000000,538000.0,3.0,2.25,2570.0,7242,2,0,0,...,2170,400,1951,1991,98125,47.7210,-122.319,1690.0,7639.0,1


#18. Create a normalized feature matrix from the TRAINING data with the following set of features.
	•	bedrooms, bathrooms, sqft_living, sqft_lot, floors, waterfront, view, condition, grade, sqft_above, sqft_basement, yr_built, yr_renovated


Make sure you store the norms for the normalization, since we’ll use them later.


In [126]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

In [145]:
my_output = 'price'
(train_feature_matrix, train_output) = get_numpy_data(train_data, all_features, my_output)
train_feature_matrix.head(2)
print (train_feature_matrix.columns.tolist())

#np.linalg.norm(train_feature_matrix, axis=0)
# normalize features
normalized_train_feature_matrix, train_norms = normalize_features(train_feature_matrix)

['constant', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated']


In [147]:
print train_norms
normalized_train_feature_matrix.head(2)

[  1.31848398e+02   4.60040216e+02   2.96850552e+02   2.99962419e+05
   5.81709718e+06   2.09458827e+02   1.15325626e+01   1.05933942e+02
   4.57793622e+02   1.02101959e+03   2.59726472e+05   7.01224951e+04
   2.59922094e+05   5.36953839e+04]


,constant,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated
0,0.007584,0.006521,0.003369,0.003934,0.000971,0.004774,0.0,0.0,0.006553,0.006856,0.004543,0.000000,0.007521,0.00000
1,0.007584,0.006521,0.007580,0.008568,0.001245,0.009548,0.0,0.0,0.006553,0.006856,0.008355,0.005704,0.007506,0.03708


#19. First, learn the weights with l1_penalty=1e7, on the training data. 

Initialize weights to all zeros, and set the tolerance=1. 

Call resulting weights’ weights1e7’, you will need them later.


In [148]:
l1_penalty=1e7
initial_weights=np.zeros(14)
tolerance=1.
weights1e7 = lasso_cyclical_coordinate_descent(normalized_train_feature_matrix, train_output,
                                            initial_weights, l1_penalty, tolerance)

In [149]:
#20. Quiz Question: What features had non-zero weight in this case?
zip(weights1e7,all_features)

[(24429597.967274334, 'bedrooms'),
 (0.0, 'bathrooms'),
 (0.0, 'sqft_living'),
 (48389176.906467639, 'sqft_lot'),
 (0.0, 'floors'),
 (0.0, 'waterfront'),
 (3317511.2178915944, 'view'),
 (7329961.6718058065, 'condition'),
 (0.0, 'grade'),
 (0.0, 'sqft_above'),
 (0.0, 'sqft_basement'),
 (0.0, 'yr_built'),
 (0.0, 'yr_renovated')]

#21. Next, learn the weights with l1_penalty=1e8, on the training data. Initialize weights to all zeros, and set the tolerance=1. Call resulting weights ‘weights1e8’, you will need them later.


In [150]:
l1_penalty=1e8
initial_weights=np.zeros(14)
tolerance=1.
weights1e8 = lasso_cyclical_coordinate_descent(normalized_train_feature_matrix, train_output,
                                            initial_weights, l1_penalty, tolerance)

In [151]:
#22. Quiz Question: What features had non-zero weight in this case?
zip(weights1e8,all_features)

[(71114625.714887127, 'bedrooms'),
 (0.0, 'bathrooms'),
 (0.0, 'sqft_living'),
 (0.0, 'sqft_lot'),
 (0.0, 'floors'),
 (0.0, 'waterfront'),
 (0.0, 'view'),
 (0.0, 'condition'),
 (0.0, 'grade'),
 (0.0, 'sqft_above'),
 (0.0, 'sqft_basement'),
 (0.0, 'yr_built'),
 (0.0, 'yr_renovated')]

#23. Finally, learn the weights with l1_penalty=1e4, on the training data. Initialize weights to all zeros, and set the tolerance=5e5. Call resulting weights ‘weights1e4’, you will need them later. (This case will take quite a bit longer to converge than the others above.)


In [152]:
l1_penalty=1e4
initial_weights=np.zeros(14)
tolerance=5e5
weights1e4 = lasso_cyclical_coordinate_descent(normalized_train_feature_matrix, train_output,
                                            initial_weights, l1_penalty, tolerance)

In [153]:
#24. Quiz Question: What features had non-zero weight in this case?
zip(weights1e4,all_features)
# all are non-zero

[(110108247.25543687, 'bedrooms'),
 (-18566779.407088313, 'bathrooms'),
 (1268310.1478815561, 'sqft_living'),
 (177305928.75502616, 'sqft_lot'),
 (-1675988.7862960447, 'floors'),
 (-142663.57652357232, 'waterfront'),
 (6713978.2808444947, 'view'),
 (5764679.5051361788, 'condition'),
 (19200097.682212196, 'grade'),
 (95947134.606725365, 'sqft_above'),
 (-102015970.88856781, 'sqft_basement'),
 (-26288607.230862282, 'yr_built'),
 (-193333306.90927121, 'yr_renovated')]

# Rescaling learned weights
#25. Recall that we normalized our feature matrix, before learning the weights. To use these weights on a test set, we must normalize the test data in the same way. Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data: 

In this case, we must scale the resulting weights so that we can make predictions with original features:
	
    •	Store the norms of the original features to a vector called ‘norms’:
            features, norms = normalize_features(features)
    
    •	Run Lasso on the normalized features and obtain a ‘weights’ vector
	
    •	Compute the weights for the original features by performing element-wise division, i.e.


weights_normalized = weights / norms

Now, we can apply weights_normalized to the test data, without normalizing it!


#26. Create a normalized version of each of the weights learned above. (‘weights1e4’, ‘weights1e7’, ‘weights1e8’). 

To check your results, if you call ‘normalized_weights1e7’ the normalized version of ‘weights1e7’, then

print normalized_weights1e7[3]
should return 161.31745624837794.


In [154]:
normalized_weights1e4 = weights1e4 / train_norms
normalized_weights1e7 = weights1e7 / train_norms
normalized_weights1e8 = weights1e8 / train_norms


In [155]:
print normalized_weights1e7[3]

161.317464763


# Evaluating each of the learned models on the test data

#27. Let's now evaluate the three models on the test data. Extract the feature matrix and output array from the TEST set. But this time, do NOT normalize the feature matrix. Instead, use the normalized version of weights to make predictions.
Compute the RSS of each of the three normalized weights on the (unnormalized) feature matrix.


In [156]:
(test_feature_matrix, test_output) = get_numpy_data(test_data, all_features, 'price')

In [158]:
#Compute the RSS of each of the three normalized weights on the (unnormalized) test_feature_matrix:

prediction=predict_output(test_feature_matrix,normalized_weights1e8)
RSS1 = ((prediction-test_output)**2).sum()
RSS1

537166151497322.4

In [159]:
prediction=predict_output(test_feature_matrix,normalized_weights1e7)
RSS2 = ((prediction-test_output)**2).sum()
RSS2

275962071782588.94

In [160]:
prediction=predict_output(test_feature_matrix,normalized_weights1e4)
RSS3 = ((prediction-test_output)**2).sum()
RSS3

206616429568999.75

In [161]:
#Which model performed best on the test data?

min(RSS1,RSS2,RSS3)

206616429568999.75